# Trabalho Prático 2  #
#### André Freitas PG54707 ####
#### Bruna Macieira PG54467 ####

## Exercício 3

O algoritmo de Boneh e Franklin (BF) discutido no +Capítulo 5b:  Curvas Elípticas e sua Aritmética é uma tecnica fundamental na chamada “Criptografia Orientada à Identidade”. Seguindo as orientações definidas nesse texto, pretende-se construir usando Sagemath uma classe Python que implemente este criptosistema.

In [12]:
%pip install pycrypto
%pip install sagemath-standard
import math
from sage.all import *
from Crypto.Hash import SHA512

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
class BonehFranklinIBE:
    def Hash(hash_input):
        hash = SHA512.new()
        str_val = str(hash_input)
        byte_val = str_val.encode()
        hash.update(byte_val)
        hexadecimal = hash.hexdigest()
        return int(hexadecimal, 16)
    
    #Fast modular exponentiation algorithm (needed for F_{p^2})
    def fastPowerSmall(g,A):
        a = g
        b = 1
        if g == 0 and A == 0:
            return "Undefined"
        else:
            while A > 0:
                if A%2 == 1:
                    b = b*a
                a = a^2
                A = A//2
                print
            return b
        
    def textToInt(w):
        n  = 0
        counter = 0
        #counter will give us the index of each character in the string
        for i in w:
            n = n + ord(i)*(256**counter)
            counter = counter + 1
        return n

    def intToText(n):
        str = ""
        while n > 0:
            a0 = n%256
            str = str + chr(a0) #chr undoes ord. ord() inputs character and outputs integer. str inputs integer between 0 and 255 and outputs character.
            n = n//256 #This is the quotient after dividing n by 256
        return str
    
    def Xor(a, b):
        int_a = int(a)
        int_b = int(b)
        return int_a ^^ int_b
    
    def is_in_prime_field(element, prime):
        return 0 <= element < prime
    
    #This defines a rational function g(x,y) on E whose divisor is div(g) = [P] + [Q] - [P+Q] - [O]

    #A, B coefficients of E. Use A,B = E.a_invariants()[3], E.a_invariants()[4]
    #P = E([xP, yP])

    def g(P,Q,x,y,E):
        positive_infinity = math.inf
        A,B = E.a_invariants()[3], E.a_invariants()[4]
        if P == E(0) or Q == E(0):
            return "no divisor"
        xP,yP = P[0],P[1]
        xQ,yQ = Q[0],Q[1]
        #Calculate slope of line connecting P and Q
        #JUST check if equal
        if yP == -yQ and xP == xQ:
            slope = +positive_infinity #symbol for Infinity
        elif P == Q:
            slope = (3*(xP**2) + A)/(2*yP)
        else:
            slope = (yQ - yP)/(xQ - xP)
        #return the function on E
        if slope == +positive_infinity:
            return x - xP
        else:
            return (y - yP - slope*(x - xP))/(x + xP + xQ - slope**2)
        
    def MillerAlgorithm(P,m,x,y,E):
        A,B = E.a_invariants()[3], E.a_invariants()[4]
        xP,yP = P[0],P[1]
        binary = m.digits(2) #gives number in binary
        n = len(binary) #trying to find what "n" is.
        T = P
        f = 1
        for i in range(n-2,-1,-1): #Stop once i = -1, so last number is 0...range(start, stop, step)
            f = (f**2)*BonehFranklinIBE.g(T,T,x,y,E)
            T *= 2 #T = 2T
            if binary[i] == 1:
                f = f*BonehFranklinIBE.g(T,P,x,y,E)
                T += P
        return f 

    def WeilPairing(P,Q,m,E):
        A,B = E.a_invariants()[3], E.a_invariants()[4]
        S = E.random_element()
        while m*S == E(0):
            S = E.random_element() #Pick point S that is not m-torsion. This guarantees that S isn't a linear combination of P and Q.
        xS,yS = S[0],S[1]
        QplusS = Q + S
        f_P_QplusS = BonehFranklinIBE.MillerAlgorithm(P,m,QplusS[0],QplusS[1],E)
        f_P_S = BonehFranklinIBE.MillerAlgorithm(P,m,xS,yS,E)
        num = f_P_QplusS/f_P_S

        PminusS = P - S
        f_Q_PminusS = BonehFranklinIBE.MillerAlgorithm(Q,m,PminusS[0],PminusS[1],E) #This is f_Q(P-S)
        f_Q_minusS = BonehFranklinIBE.MillerAlgorithm(Q,m,xS,-yS,E) #This is f_Q(-S)
        denom = f_Q_PminusS/f_Q_minusS

        e_m = num/denom
        return e_m
    
    def ModifiedWeilPairing(P,Q,m,E):
        Fp = GF(p)
        R.<x> = Fp[]
        Fp2.<z> = Fp.extension(x^2+x+1)      #Form F_{p^2} by adjoining z = {a nontrivial cube root of 1}
        E_zeta = EllipticCurve(Fp2, [0,1])   #Define E: y^2 = x^3 + 1 over this field
        phiQ = E_zeta([z*Q[0],Q[1]])
        A,B = E_zeta.a_invariants()[3], E_zeta.a_invariants()[4]
        P_zeta = E_zeta([P[0],P[1]])
        S = E_zeta.random_element()
        while m*S == E(0):
            S = E_zeta.random_element()
        xS,yS = S[0],S[1]
        QplusS = phiQ + S
        f_P_QplusS = BonehFranklinIBE.MillerAlgorithm(P,m,QplusS[0],QplusS[1],E_zeta)
        f_P_S = BonehFranklinIBE.MillerAlgorithm(P,m,xS,yS,E_zeta)
        num = f_P_QplusS/f_P_S

        PminusS = P_zeta - S #modify
        f_Q_PminusS = BonehFranklinIBE.MillerAlgorithm(phiQ,m,PminusS[0],PminusS[1],E_zeta) #This is f_Q(P-S)
        f_Q_minusS = BonehFranklinIBE.MillerAlgorithm(phiQ,m,xS,-yS,E_zeta) #This is f_Q(-S)
        denom = f_Q_PminusS/f_Q_minusS

        e_m = num/denom
        return e_m
    
    def KeyGen(k):
        q = random_prime((2^k) - 1, True, lbound=2^(k-1)) #Generates a random k-bit prime. False means using pseudo-primality tests.
        p = q
        l = 1 #need l for `KeyExtract`
        lq = q
        while True:
            l += 1
            lq += q
            p = lq - 1
            if p%3 == 2 and (p+1)%(q^2) != 0 and is_prime(p) == True:
                break
        E = EllipticCurve(GF(p), [0,1]) #p is public b/c the elliptic curve is known
        P = E(0)
        while P == E(0):
            Q = E.random_element()
            while Q == E(0): #make sure P is not O
                Q = E.random_element()
            h = (p+1)//q #This is to make sure P has order q.
            P = h*Q #Order of P is q1
        s = ZZ.random_element(2,q-1) #s is private master key in Z_q*.
        P_pub = s*P
        params = [p, q, l, E, P, P_pub]
        return params, s 
    
    def KeyExtract(y0, params):
        p, q, l, E, P, P_pub = params
        x0 = pow((y0^2) - 1, ((2*p)-1)//3, p) #`pow` is Python's built-in function that does fast power
        Q = E([x0,y0])
        Q_ID = l*Q #l comes from BDHGenerator. It's the integer s.t. p = lq-1
        d_ID = s*Q_ID
        return Q_ID,d_ID
    
    def Encrypt(M,Q_ID,params):
        params = [p, q, l, E, P, P_pub]
        r = ZZ.random_element(2,q-1)
        U = r*P
        g_ID = BonehFranklinIBE.ModifiedWeilPairing(Q_ID,P_pub,q,E)
        g_ID_to_r = BonehFranklinIBE.fastPowerSmall(g_ID, r)
        V = M^^BonehFranklinIBE.Hash(g_ID_to_r)
        C = [U,V] #This is the ciphertext
        return C
    
    def Decrypt(C,d_ID,params):
        params = [p, q, l, E, P, P_pub]
        U,V = C
        weil = BonehFranklinIBE.ModifiedWeilPairing(d_ID,U,q,E)
        M = V^^BonehFranklinIBE.Hash(weil)
        return M

In [14]:
#Step 1: Only need to run once
params,s = BonehFranklinIBE.KeyGen(512)
p, q, l, E, P, P_pub = params
print("p = ", p)
print("q = ", q)
print("E = ", E)
print("P = ", P)
print("s = ", s)
print("l = ", l)
print("P_pub = ", P_pub)

p =  962846820577836932489159090110661220757528168191097735105207174340143623541384446207404899892679199412178001546662332785896486771705856151963259065245943159
q =  8023723504815307770742992417588843506312734734925814459210059786167863529511537051728374165772326661768150012888852773215804056430882134599693825543716193
E =  Elliptic Curve defined by y^2 = x^3 + 1 over Finite Field of size 962846820577836932489159090110661220757528168191097735105207174340143623541384446207404899892679199412178001546662332785896486771705856151963259065245943159
P =  (226195724914835089074744301684146367700203692735586370195430450590265336487643161194551299457036867499561155324558463043104951894676581482156695750941645083 : 741160221313849950339830941899531557164987627588079120179174157814124567299669049187945749064637781213486008027992700883822803381008006147030188861402088341 : 1)
s =  7939768147232293093365257803835054728165721789754580870999590544258967266817158979443694291781272885606846350738561142

In [15]:
#Step 2: Hash. Only need to run once for one ID.
your_ID = "user"

y0 = BonehFranklinIBE.Hash(your_ID)
print("y0 = ", y0)

y0 =  9284027244711476868736574711199185332133782967237044148153727596027875067278862165519633211530008452951448644427085930313537133830441524639977514866580706


In [16]:
#Step 3: Only need to run once.
Q_ID,d_ID = BonehFranklinIBE.KeyExtract(y0,params)
print("Q_ID = ", Q_ID)
print("d_ID = ", d_ID)

Q_ID =  (944061521927993948104389957199814315096524917885333783729896465072754917830787084151472781207319482245362486285164499741136049402624393942205124220532679264 : 224550263820215285952021615872355505844798422481148826679335278523113890028112939825826866515307921971003174320556560877965216074744056607463774748678324824 : 1)
d_ID =  (681753144963497700683164139653618668955291497477322136227461699276247377378296286112026724067336569696481393944239135407949163771186309923247891499892642410 : 792530164652065234266656230155957453983750394639173005606043237946191376471506260605223530519174072610590732915970683420810235277590053936730815032665795269 : 1)


In [17]:
#Step 4: Encrypt
input_message = "Teste"

M = BonehFranklinIBE.textToInt(input_message)
C = BonehFranklinIBE.Encrypt(M,Q_ID,params)
U,V = C
print("M = ", M)
print("U = ", U)
print("V = ", V)

M =  435745416532
U =  (221137235510804569941339894848401657821626002890018091088704097374189688039809814191299299334186753104833287841636556344758619842290975496814606416213258969 : 516246160207303673390014545982468056610365291226760573710237237884524001759337339406773946541732296425277827305684333913987168329818758266478507204544801094 : 1)
V =  8624699354936482274031475692361212346275005300788943141442322296349914466108087418462263424991376777202841830610872544235222853911574722902141719611180014


In [18]:
#Step 5: Decrypt
M0 = BonehFranklinIBE.Decrypt(C,d_ID,params)
output_message = BonehFranklinIBE.intToText(M0)
print("M0 = ", M0)
print("messsage = ", output_message)

M0 =  435745416532
messsage =  Teste
